In [2]:
import os
from dotenv import load_dotenv
from langchain_aws import ChatBedrockConverse
from typing import TypedDict, List, Union
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, AIMessage

load_dotenv()

llm = ChatBedrockConverse(
    model_id="us.anthropic.claude-3-5-haiku-20241022-v1:0",
    region_name="us-east-1",
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    aws_session_token=os.getenv("AWS_SESSION_TOKEN")
)

# create a form of memory for the agent
class AgentState(TypedDict):
    messages: List[Union[HumanMessage, AIMessage]]


# Defining Node
def process(state: AgentState) -> AgentState:
    response = llm.invoke(state["messages"])
    state["messages"].append(AIMessage(content=response.content))
    print(f"\nAI: {response.content}")
    print(f"Current State: {state['messages']}")


graph = StateGraph(AgentState)
graph.add_node("process", process)
graph.add_edge(START, "process")
graph.add_edge("process", END)
agent = graph.compile()

conversation_history = []

user_input = input("Enter: ")
while user_input != "exit":
    conversation_history.append(HumanMessage(content=user_input))
    start_state = AgentState(messages=conversation_history)
    result = agent.invoke(start_state)
    conversation_history = result["messages"]
    user_input = input("Enter: ")

with open("logging.txt", "w") as file:
    file.write("Conversation Log:\n")
    for message in conversation_history:
        if isinstance(message, HumanMessage):
            file.write(f"You: {message.content}\n")
        elif isinstance(message, AIMessage):
            file.write(f"AI: {message.content}\n")
print("Converstation saved in logging.txt file")


AI: Hello! How are you doing today? Is there anything I can help you with?
Current State: [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! How are you doing today? Is there anything I can help you with?', additional_kwargs={}, response_metadata={})]

AI: Here's a 20-word essay about AI agents:

AI agents are intelligent systems that perceive environments, make decisions, and take actions to achieve specific goals autonomously and adaptively.
Current State: [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! How are you doing today? Is there anything I can help you with?', additional_kwargs={}, response_metadata={}), HumanMessage(content='white me a short essay in 20 words about AI agent', additional_kwargs={}, response_metadata={}), AIMessage(content="Here's a 20-word essay about AI agents:\n\nAI agents are intelligent systems that perceive environments, make decisions, and take actions to